# LangChain RAG (Retrieval-Augmented Generation) - Complete Guide

## Overview

This notebook provides a comprehensive guide to building a **Retrieval-Augmented Generation (RAG)** system using LangChain. RAG is a powerful technique that combines the strengths of large language models (LLMs) with external knowledge retrieval.

### What is RAG?

RAG enhances LLM responses by:
1. **Retrieving** relevant documents from a knowledge base
2. **Augmenting** the prompt with retrieved context
3. **Generating** informed responses based on both the LLM's knowledge and retrieved documents

### RAG Architecture

```
User Query → Embedding → Vector Search → Retrieved Docs → LLM → Response
                ↓                          ↓
         Vector Store ← Embeddings ← Document Chunks ← Documents
```

### What You'll Learn

- Document loading and preprocessing
- Text splitting strategies
- **OpenAI vs HuggingFace embeddings** (with comparisons)
- Vector store creation with FAISS
- Different retrieval strategies (Similarity vs MMR)
- RAG chain construction
- Performance evaluation and best practices

---

## 1. Setup and Installation

First, we'll install all required dependencies. This includes:
- **langchain**: Core framework
- **langchain-community**: Community integrations
- **langchain-openai**: OpenAI integrations
- **langchain-huggingface**: HuggingFace integrations
- **openai**: OpenAI API client
- **faiss-cpu**: Vector similarity search
- **tiktoken**: Token counting
- **sentence-transformers**: For HuggingFace embeddings

In [1]:
# Install required packages
%pip install -q langchain langchain-community langchain-openai langchain-huggingface openai faiss-cpu tiktoken sentence-transformers
%pip install -q beautifulsoup4 python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Configure API Keys

**Security Best Practice**: Never hardcode API keys in your notebooks. Use environment variables or secure secret management.

For Google Colab, use the `userdata` feature. For local environments, use a `.env` file or environment variables.

In [ ]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Suppress HuggingFace tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load OpenAI API key (REQUIRED for OpenAI embeddings and LLM)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# NOTE: HuggingFace API key NOT needed for this notebook
# We use local sentence-transformers embeddings which run completely offline
# 
# HuggingFace API key would only be needed for:
# - HuggingFace Inference API (paid API service)
# - Gated models requiring authentication
# - Higher rate limits on model downloads from Hub

In [ ]:
# Verify that OpenAI API key is loaded correctly
print("=" * 60)
print("API KEY VERIFICATION")
print("=" * 60)

if OPENAI_API_KEY:
    print(f"\n✓ OpenAI API Key: LOADED")
    print(f"  Preview: {OPENAI_API_KEY[:7]}...{OPENAI_API_KEY[-4:]}")
else:
    print("\n✗ OpenAI API Key: NOT LOADED")
    print("\n⚠️  WARNING: OPENAI_API_KEY is required!")
    print("  1. Create .env file in project root")
    print("  2. Add: OPENAI_API_KEY=your-key-here")
    print("  3. Get key from: https://platform.openai.com/api-keys")
    print("  4. Restart kernel after updating .env")

print("\n" + "=" * 60)
print("ℹ️  HuggingFace Embeddings:")
print("  - No API key needed (runs locally)")
print("  - Uses sentence-transformers library")
print("  - Downloads models to ~/.cache/huggingface/")
print("=" * 60)

In [4]:
# Test the OpenAI API key directly
from openai import OpenAI

print("Testing OpenAI API key...")
try:
    client = OpenAI(api_key=OPENAI_API_KEY)
    # Try a simple API call
    response = client.models.list()
    print("✓ API key is VALID! Connection successful.")
    print(f"  Available models: {len(list(response.data))} models found")
except Exception as e:
    print("✗ API key is INVALID!")
    print(f"  Error: {str(e)}")
    print("\n⚠️  Please verify your OpenAI API key:")
    print("  1. Go to https://platform.openai.com/api-keys")
    print("  2. Create a new API key")
    print("  3. Update the .env file with the new key")
    print("  4. Restart the kernel and rerun from the beginning")

Testing OpenAI API key...
✓ API key is VALID! Connection successful.
  Available models: 101 models found
✓ API key is VALID! Connection successful.
  Available models: 101 models found


---

## 2. Document Loading

The first step in building a RAG system is loading documents. LangChain supports various document loaders:
- **WebBaseLoader**: Load content from web pages
- **PyPDFLoader**: Load PDF files
- **TextLoader**: Load plain text files
- **DirectoryLoader**: Load multiple files from a directory

In this example, we'll load LangChain documentation pages about RAG and related topics.

In [5]:
from langchain_community.document_loaders import WebBaseLoader
import datetime

# Define URLs for LangChain documentation on RAG
urls = [
    "https://python.langchain.com/docs/use_cases/question_answering/",
    "https://python.langchain.com/docs/modules/data_connection/retrievers/",
    "https://python.langchain.com/docs/modules/model_io/llms/",
    "https://python.langchain.com/docs/use_cases/chatbots/"
]

# Initialize WebBaseLoader and load documents
print("Loading documents from web...")
loader = WebBaseLoader(urls)
docs = loader.load()
print(f"✓ Loaded {len(docs)} documents")

# Add custom metadata to documents
# This is useful for filtering and source attribution
current_date = datetime.date.today().isoformat()
for doc in docs:
    doc.metadata['source_type'] = 'web_documentation'
    doc.metadata['process_date'] = current_date
    doc.metadata['domain'] = 'langchain'

print("✓ Added custom metadata to all documents")

# Display first document info
if docs:
    print("\n--- First Document ---")
    print(f"Source: {docs[0].metadata.get('source', 'N/A')}")
    print(f"Content preview (first 300 chars):\n{docs[0].page_content[:300]}...")
    print(f"\nMetadata: {docs[0].metadata}")

/Users/gianlucamazza/Workspace/notebooks/llm_rag/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
USER_AGENT environment variable not set, consider setting it to identify your requests.
USER_AGENT environment variable not set, consider setting it to identify your requests.


Loading documents from web...
✓ Loaded 4 documents
✓ Added custom metadata to all documents

--- First Document ---
Source: https://python.langchain.com/docs/use_cases/question_answering/
Content preview (first 300 chars):
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

Metadata: {'source': 'https://python.langchain.com/docs/use_cases/question_answering/', 'title': 'Build a RAG agent with LangChain - Docs by LangChain', 'language': 'en', 'source_type': 'web_documentation', 'process_date': '2025-11-12', 'domain': 'langchain'}
✓ Loaded 4 documents
✓ Added custom metadata to all documents

--- First Document ---
Source: https://python.langchain.com/docs/use_cases/question_answering/
Content preview (first 300 chars):
Build a RAG agent w

---

## 3. Text Splitting Strategies

Large documents must be split into smaller chunks for effective retrieval. The key parameters are:

- **chunk_size**: Maximum number of characters per chunk
- **chunk_overlap**: Number of overlapping characters between chunks

### Why Overlap Matters

Overlap ensures that context isn't lost at chunk boundaries. For example, if a sentence is split between two chunks, overlap helps preserve its meaning.

### Strategy Comparison

We'll compare two strategies:
1. **Strategy A**: chunk_size=1000, chunk_overlap=200 (better for longer context)
2. **Strategy B**: chunk_size=500, chunk_overlap=100 (better for precise retrieval)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Strategy A: Larger chunks with more overlap
print("=== Strategy A: Larger Chunks ===")
text_splitter_a = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks_a = text_splitter_a.split_documents(docs)
print(f"Created {len(chunks_a)} chunks with chunk_size=1000, chunk_overlap=200")

# Strategy B: Smaller chunks with less overlap
print("\n=== Strategy B: Smaller Chunks ===")
text_splitter_b = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks_b = text_splitter_b.split_documents(docs)
print(f"Created {len(chunks_b)} chunks with chunk_size=500, chunk_overlap=100")

# Compare strategies
print("\n=== Comparison ===")
print(f"Strategy A: {len(chunks_a)} chunks (fewer, longer chunks)")
print(f"Strategy B: {len(chunks_b)} chunks (more, shorter chunks)")
print(f"Ratio: {len(chunks_b) / len(chunks_a):.2f}x more chunks with Strategy B")

# Display sample chunks
print("\n--- Strategy A - Sample Chunk ---")
print(f"Length: {len(chunks_a[0].page_content)} chars")
print(f"Content: {chunks_a[0].page_content[:300]}...")

print("\n--- Strategy B - Sample Chunk ---")
print(f"Length: {len(chunks_b[0].page_content)} chars")
print(f"Content: {chunks_b[0].page_content[:300]}...")

# We'll use Strategy A for the rest of the notebook
chunks = chunks_a
print(f"\n✓ Using Strategy A ({len(chunks)} chunks) for subsequent examples")

=== Strategy A: Larger Chunks ===
Created 120 chunks with chunk_size=1000, chunk_overlap=200

=== Strategy B: Smaller Chunks ===
Created 256 chunks with chunk_size=500, chunk_overlap=100

=== Comparison ===
Strategy A: 120 chunks (fewer, longer chunks)
Strategy B: 256 chunks (more, shorter chunks)
Ratio: 2.13x more chunks with Strategy B

--- Strategy A - Sample Chunk ---
Length: 839 chars
Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

--- Strategy B - Sample Chunk ---
Length: 498 chars
Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSm

---

## 4. Embeddings: OpenAI vs HuggingFace

Embeddings convert text into numerical vectors that capture semantic meaning. Similar texts have similar vector representations.

### Comparison

| Feature | OpenAI Embeddings | HuggingFace Embeddings |
|---------|-------------------|------------------------|
| **Cost** | Pay per token | Free (local) |
| **Speed** | Fast (API) | Slower (local compute) |
| **Quality** | Very high | Good to high (model-dependent) |
| **Privacy** | Data sent to OpenAI | Data stays local |
| **Internet** | Required | Not required |

### When to Use Each

- **OpenAI**: Production systems, high quality needed, budget available
- **HuggingFace**: Privacy-sensitive data, cost constraints, offline operation

### 4.1 OpenAI Embeddings

OpenAI's `text-embedding-3-small` model provides high-quality embeddings with good performance.

In [7]:
from langchain_openai import OpenAIEmbeddings
import time

print("Initializing OpenAI Embeddings...")
openai_embeddings = OpenAIEmbeddings()
print("✓ OpenAI Embeddings initialized")

# Test embedding generation
test_text = "What is retrieval-augmented generation?"
start_time = time.time()
test_embedding = openai_embeddings.embed_query(test_text)
elapsed = time.time() - start_time

print(f"\nTest Query: '{test_text}'")
print(f"Embedding dimension: {len(test_embedding)}")
print(f"Time taken: {elapsed:.3f}s")
print(f"First 5 values: {test_embedding[:5]}")

Initializing OpenAI Embeddings...
✓ OpenAI Embeddings initialized

Test Query: 'What is retrieval-augmented generation?'
Embedding dimension: 1536
Time taken: 0.828s
First 5 values: [-0.040319960564374924, -0.0035607097670435905, 5.0805494538508356e-05, 0.000940056168474257, -0.010274112224578857]

Test Query: 'What is retrieval-augmented generation?'
Embedding dimension: 1536
Time taken: 0.828s
First 5 values: [-0.040319960564374924, -0.0035607097670435905, 5.0805494538508356e-05, 0.000940056168474257, -0.010274112224578857]


### 4.2 HuggingFace Embeddings

We'll use `sentence-transformers/all-MiniLM-L6-v2`, a popular open-source model that provides good quality embeddings with reasonable speed.

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

print("Initializing HuggingFace Embeddings...")
print("(First run will download the model - this may take a minute)")
print(f"Model cache location: {os.path.expanduser('~/.cache/huggingface/')}\n")

try:
    hf_embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    print("✓ HuggingFace Embeddings initialized successfully")
    
    # Test embedding generation
    start_time = time.time()
    test_embedding_hf = hf_embeddings.embed_query(test_text)
    elapsed_hf = time.time() - start_time
    
    print(f"\nTest Query: '{test_text}'")
    print(f"Embedding dimension: {len(test_embedding_hf)}")
    print(f"Time taken: {elapsed_hf:.3f}s")
    print(f"First 5 values: {test_embedding_hf[:5]}")
    
except Exception as e:
    print(f"✗ Error initializing HuggingFace embeddings: {e}")
    print("\n⚠️  Troubleshooting steps:")
    print("  1. Check internet connection (needed for first download)")
    print("  2. Verify disk space: ~200MB free required")
    print("  3. Try: pip install --upgrade sentence-transformers")
    print("  4. Check model name is correct: 'sentence-transformers/all-MiniLM-L6-v2'")
    print("  5. Clear cache if corrupted: rm -rf ~/.cache/huggingface/")
    raise

Initializing HuggingFace Embeddings...
(First run will download the model - this may take a minute)
Model cache location: /Users/gianlucamazza/.cache/huggingface/

✓ HuggingFace Embeddings initialized successfully
✓ HuggingFace Embeddings initialized successfully

Test Query: 'What is retrieval-augmented generation?'
Embedding dimension: 384
Time taken: 2.561s
First 5 values: [-0.11101917177438736, -0.026313453912734985, -0.05789430812001228, 0.05978527292609215, -0.020831570029258728]

Test Query: 'What is retrieval-augmented generation?'
Embedding dimension: 384
Time taken: 2.561s
First 5 values: [-0.11101917177438736, -0.026313453912734985, -0.05789430812001228, 0.05978527292609215, -0.020831570029258728]


### 4.3 Side-by-Side Comparison

Let's compare the two embedding approaches with the same test query.

In [9]:
import numpy as np

print("=== Embeddings Comparison ===")
print(f"\nTest Query: '{test_text}'\n")

comparison_data = [
    ["Feature", "OpenAI", "HuggingFace"],
    ["Dimension", len(test_embedding), len(test_embedding_hf)],
    ["Time (s)", f"{elapsed:.3f}", f"{elapsed_hf:.3f}"],
    ["Mean value", f"{np.mean(test_embedding):.4f}", f"{np.mean(test_embedding_hf):.4f}"],
    ["Std dev", f"{np.std(test_embedding):.4f}", f"{np.std(test_embedding_hf):.4f}"]
]

# Print comparison table
col_widths = [max(len(str(row[i])) for row in comparison_data) + 2 for i in range(3)]
for i, row in enumerate(comparison_data):
    print("".join(str(item).ljust(col_widths[j]) for j, item in enumerate(row)))
    if i == 0:
        print("-" * sum(col_widths))

print("\n💡 Key Takeaway:")
print("   - OpenAI: Higher dimension (1536), typically higher quality")
print("   - HuggingFace: Lower dimension (384), faster and free")

=== Embeddings Comparison ===

Test Query: 'What is retrieval-augmented generation?'

Feature     OpenAI   HuggingFace  
----------------------------------
Dimension   1536     384          
Time (s)    0.828    2.561        
Mean value  -0.0008  0.0002       
Std dev     0.0255   0.0510       

💡 Key Takeaway:
   - OpenAI: Higher dimension (1536), typically higher quality
   - HuggingFace: Lower dimension (384), faster and free


---

## 5. Vector Store Creation

Vector stores enable efficient similarity search over embeddings. We use **FAISS** (Facebook AI Similarity Search), which provides:
- Fast similarity search
- Efficient memory usage
- Support for large-scale datasets

We'll create two vector stores to compare both embedding approaches.

### 5.1 Vector Store with OpenAI Embeddings

In [10]:
from langchain_community.vectorstores import FAISS

print("Creating FAISS vector store with OpenAI embeddings...")
start_time = time.time()

vectorstore_openai = FAISS.from_documents(chunks, openai_embeddings)

elapsed = time.time() - start_time
print(f"✓ Vector store created in {elapsed:.2f}s")
print(f"  - {len(chunks)} documents indexed")
print("  - Embedding dimension: 1536")

Creating FAISS vector store with OpenAI embeddings...
✓ Vector store created in 0.95s
  - 120 documents indexed
  - Embedding dimension: 1536
✓ Vector store created in 0.95s
  - 120 documents indexed
  - Embedding dimension: 1536


### 5.2 Vector Store with HuggingFace Embeddings

In [11]:
print("Creating FAISS vector store with HuggingFace embeddings...")
start_time = time.time()

vectorstore_hf = FAISS.from_documents(chunks, hf_embeddings)

elapsed = time.time() - start_time
print(f"✓ Vector store created in {elapsed:.2f}s")
print(f"  - {len(chunks)} documents indexed")
print("  - Embedding dimension: 384")

Creating FAISS vector store with HuggingFace embeddings...
✓ Vector store created in 2.15s
  - 120 documents indexed
  - Embedding dimension: 384
✓ Vector store created in 2.15s
  - 120 documents indexed
  - Embedding dimension: 384


### 5.3 Vector Store Persistence (Best Practice)

**Production Best Practice**: Save vector stores to avoid recomputing embeddings.

Benefits:
- **Skip re-embedding** on kernel restart (saves time and API costs)
- **Share indexes** across team members
- **Version control** your vector stores
- **Faster development** iteration

In [12]:
# Example: Save vector stores for future use
import os

# Create directory for vector stores
os.makedirs("./vector_stores", exist_ok=True)

print("💾 Saving vector stores...")

# Save OpenAI vector store
vectorstore_openai.save_local("./vector_stores/faiss_openai")
print("✓ Saved OpenAI vector store to ./vector_stores/faiss_openai")

# Save HuggingFace vector store  
vectorstore_hf.save_local("./vector_stores/faiss_hf")
print("✓ Saved HuggingFace vector store to ./vector_stores/faiss_hf")

print("\n📂 Vector stores saved! To load later, use:")
print("""
# Load OpenAI vector store
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings()
vectorstore_openai = FAISS.load_local(
    "./vector_stores/faiss_openai",
    openai_embeddings,
    allow_dangerous_deserialization=True  # Required for pickle files
)

# Load HuggingFace vector store
from langchain_huggingface import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore_hf = FAISS.load_local(
    "./vector_stores/faiss_hf",
    hf_embeddings,
    allow_dangerous_deserialization=True
)
""")

💾 Saving vector stores...
✓ Saved OpenAI vector store to ./vector_stores/faiss_openai
✓ Saved HuggingFace vector store to ./vector_stores/faiss_hf

📂 Vector stores saved! To load later, use:

# Load OpenAI vector store
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings()
vectorstore_openai = FAISS.load_local(
    "./vector_stores/faiss_openai",
    openai_embeddings,
    allow_dangerous_deserialization=True  # Required for pickle files
)

# Load HuggingFace vector store
from langchain_huggingface import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore_hf = FAISS.load_local(
    "./vector_stores/faiss_hf",
    hf_embeddings,
    allow_dangerous_deserialization=True
)



### 5.3 Test Similarity Search

Let's test similarity search on both vector stores to see how they perform.

In [13]:
query = "How to build a RAG agent with LangChain?"

print(f"Query: '{query}'\n")
print("=" * 80)

# Test OpenAI vector store
print("\n--- OpenAI Embeddings Results ---")
results_openai = vectorstore_openai.similarity_search(query, k=3)
for i, doc in enumerate(results_openai, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:200]}...")

# Test HuggingFace vector store
print("\n" + "=" * 80)
print("\n--- HuggingFace Embeddings Results ---")
results_hf = vectorstore_hf.similarity_search(query, k=3)
for i, doc in enumerate(results_hf, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:200]}...")

print("\n" + "=" * 80)
print("\n💡 Notice how both retrievers find relevant documents, though ordering may differ.")

Query: 'How to build a RAG agent with LangChain?'


--- OpenAI Embeddings Results ---

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + Lang...

2. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + Lang...

3. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Content: We can see the full sequence of steps, along with latency and other metadata, in the LangSmith trace.
You can add a deeper level of control and customization using the LangGraph framework directly— fo...


--- HuggingFace Embeddings Results ---

1. Source: https

---

## 6. Retrieval Strategies

Different retrieval strategies optimize for different goals:

### Similarity Search
- Returns documents most similar to the query
- Simple and fast
- May return redundant documents

### MMR (Maximal Marginal Relevance)
- Balances relevance with diversity
- Reduces redundancy in results
- Particularly useful when documents contain similar information
- Controlled by `lambda_mult` parameter:
  - `lambda_mult=1.0`: Pure relevance (like similarity search)
  - `lambda_mult=0.0`: Pure diversity
  - `lambda_mult=0.5`: Balanced (recommended)

Let's compare both strategies using the OpenAI vector store.

### 6.1 Standard Similarity Retriever

In [14]:
# Create similarity-based retriever
similarity_retriever = vectorstore_openai.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # Retrieve top 4 documents
)

print("✓ Similarity retriever created")
print("  - Search type: similarity")
print("  - Documents to retrieve: 4")

✓ Similarity retriever created
  - Search type: similarity
  - Documents to retrieve: 4


### 6.2 MMR (Maximal Marginal Relevance) Retriever

In [15]:
# Create MMR-based retriever
mmr_retriever = vectorstore_openai.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 4,              # Number of documents to return
        "fetch_k": 20,       # Number of documents to fetch before MMR
        "lambda_mult": 0.5   # Balance between relevance (1.0) and diversity (0.0)
    }
)

print("✓ MMR retriever created")
print("  - Search type: mmr")
print("  - Documents to retrieve: 4")
print("  - Fetch size: 20")
print("  - Lambda (relevance/diversity): 0.5")

✓ MMR retriever created
  - Search type: mmr
  - Documents to retrieve: 4
  - Fetch size: 20
  - Lambda (relevance/diversity): 0.5


### 6.3 Compare Retrieval Strategies

In [16]:
query = "What are the steps to build a RAG agent with LangChain?"

print(f"Query: '{query}'\n")
print("=" * 80)

# Test similarity retriever
print("\n--- Similarity Search Results ---")
similarity_docs = similarity_retriever.invoke(query)
for i, doc in enumerate(similarity_docs, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:200]}...")

# Test MMR retriever
print("\n" + "=" * 80)
print("\n--- MMR Search Results ---")
mmr_docs = mmr_retriever.invoke(query)
for i, doc in enumerate(mmr_docs, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:200]}...")

print("\n" + "=" * 80)
print("\n💡 Key Observation:")
print("   MMR results should show more diversity in sources and content")
print("   while still maintaining relevance to the query.")

Query: 'What are the steps to build a RAG agent with LangChain?'


--- Similarity Search Results ---

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + Lang...

2. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + Lang...

3. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Content: We can see the full sequence of steps, along with latency and other metadata, in the LangSmith trace.
You can add a deeper level of control and customization using the LangGraph framework directly— fo...

4. Source: https://python.langchain.com/do

---

## 7. RAG Chain Construction

Now we'll build complete RAG chains that combine:
1. **LLM**: Generates answers
2. **Retriever**: Finds relevant documents
3. **Prompt**: Structures the input
4. **Chain**: Orchestrates the flow

The chain workflow:
```
User Query → Retriever → Retrieved Docs → Prompt + LLM → Final Answer
```

### 7.1 Initialize LLM

We'll use GPT-4o-mini for cost-effectiveness. For production, consider GPT-5 for higher quality.

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # Deterministic responses
)

print("✓ ChatOpenAI LLM initialized")
print("  - Model: gpt-4o-mini")
print("  - Temperature: 0 (deterministic)")

✓ ChatOpenAI LLM initialized
  - Model: gpt-4o-mini
  - Temperature: 0 (deterministic)


### 7.2 Create Prompt Template

The prompt instructs the LLM on how to use the retrieved context.

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant. Answer the user's question based on the context provided below.
    
If the context doesn't contain enough information to answer the question, say so clearly.
Always cite which parts of the context you used to formulate your answer.

Context:
{context}"""),
    ("user", "{input}"),
])

print("✓ Prompt template created")

✓ Prompt template created


### 7.3 Build Document Chain and Retrieval Chains

We'll create:
1. A document chain that combines LLM with prompt
2. Retrieval chains for both similarity and MMR strategies

In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Create a simple RAG chain using LCEL (LangChain Expression Language)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build the similarity retrieval chain
similarity_retrieval_chain = (
    {"context": similarity_retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("✓ Similarity retrieval chain created")

# Build the MMR retrieval chain
mmr_retrieval_chain = (
    {"context": mmr_retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("✓ MMR retrieval chain created")

print("\n✓ RAG chains ready for inference")

✓ Similarity retrieval chain created
✓ MMR retrieval chain created

✓ RAG chains ready for inference


---

## 8. RAG in Action: Comparison & Evaluation

Let's test our RAG chains with various queries and compare their performance.

### 8.1 Test Query with Similarity Retrieval

In [20]:
user_query = "How to build a RAG agent with LangChain?"

print("=" * 80)
print(f"QUERY: {user_query}")
print("=" * 80)
print("\n--- SIMILARITY RETRIEVAL CHAIN ---\n")

# First, retrieve the documents using the retriever
retrieved_docs = similarity_retriever.invoke(user_query)

print("Retrieved Documents (Context):")
print("-" * 80)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:300]}...")

# Now invoke the chain to get the answer
response_similarity = similarity_retrieval_chain.invoke(user_query)

print("\n" + "=" * 80)
print("GENERATED ANSWER:")
print("=" * 80)
print(response_similarity)

QUERY: How to build a RAG agent with LangChain?

--- SIMILARITY RETRIEVAL CHAIN ---

Retrieved Documents (Context):
--------------------------------------------------------------------------------

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

2. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

3. Source: https://python.la

### 8.2 Test Same Query with MMR Retrieval

In [21]:
print("=" * 80)
print(f"QUERY: {user_query}")
print("=" * 80)
print("\n--- MMR RETRIEVAL CHAIN ---\n")

# First, retrieve the documents using the MMR retriever
retrieved_docs_mmr = mmr_retriever.invoke(user_query)

print("Retrieved Documents (Context):")
print("-" * 80)
for i, doc in enumerate(retrieved_docs_mmr, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:300]}...")

# Now invoke the chain to get the answer
response_mmr = mmr_retrieval_chain.invoke(user_query)

print("\n" + "=" * 80)
print("GENERATED ANSWER:")
print("=" * 80)
print(response_mmr)

QUERY: How to build a RAG agent with LangChain?

--- MMR RETRIEVAL CHAIN ---

Retrieved Documents (Context):
--------------------------------------------------------------------------------

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

2. Source: https://python.langchain.com/docs/modules/model_io/llms/
   Content: Advanced streaming topics"Auto-streaming" chat modelsLangChain simplifies streaming from chat models by automatically enabling streaming mode in certain cases, even when you’re not explicitly calling the streaming methods. This is particularly useful when you use the non-streaming invoke method but ...

3. Source: https://python.langch

### 8.3 Compare OpenAI vs HuggingFace Embeddings

Let's create a retrieval chain using the HuggingFace vector store and compare results.

In [22]:
# Create retriever from HuggingFace vector store
hf_retriever = vectorstore_hf.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

# Create a helper function for formatting docs (if not already defined)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create retrieval chain with HuggingFace embeddings using LCEL
hf_retrieval_chain = (
    {"context": hf_retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("=" * 80)
print(f"QUERY: {user_query}")
print("=" * 80)
print("\n--- HUGGINGFACE EMBEDDINGS CHAIN ---\n")

# First, retrieve the documents using the HF retriever
retrieved_docs_hf = hf_retriever.invoke(user_query)

print("Retrieved Documents (Context):")
print("-" * 80)
for i, doc in enumerate(retrieved_docs_hf, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content: {doc.page_content[:300]}...")

# Now invoke the chain to get the answer
response_hf = hf_retrieval_chain.invoke(user_query)

print("\n" + "=" * 80)
print("GENERATED ANSWER:")
print("=" * 80)
print(response_hf)

QUERY: How to build a RAG agent with LangChain?

--- HUGGINGFACE EMBEDDINGS CHAIN ---

Retrieved Documents (Context):
--------------------------------------------------------------------------------

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

2. Source: https://python.langchain.com/docs/use_cases/chatbots/
   Content: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with L...

3. Source: https://python.

### 8.4 Summary Comparison

In [23]:
print("=" * 80)
print("COMPARISON SUMMARY")
print("=" * 80)

print("\n1. OpenAI Embeddings + Similarity Search")
print(f"   Answer length: {len(response_similarity)} chars")
print(f"   Documents retrieved: {len(retrieved_docs)}")

print("\n2. OpenAI Embeddings + MMR Search")
print(f"   Answer length: {len(response_mmr)} chars")
print(f"   Documents retrieved: {len(retrieved_docs_mmr)}")

print("\n3. HuggingFace Embeddings + Similarity Search")
print(f"   Answer length: {len(response_hf)} chars")
print(f"   Documents retrieved: {len(retrieved_docs_hf)}")

print("\n" + "=" * 80)
print("\n💡 Key Insights:")
print("   - All approaches provide relevant answers")
print("   - MMR may provide more diverse context")
print("   - HuggingFace embeddings are competitive and free")
print("   - Choice depends on: budget, privacy needs, and quality requirements")

COMPARISON SUMMARY

1. OpenAI Embeddings + Similarity Search
   Answer length: 417 chars
   Documents retrieved: 4

2. OpenAI Embeddings + MMR Search
   Answer length: 1380 chars
   Documents retrieved: 4

3. HuggingFace Embeddings + Similarity Search
   Answer length: 1585 chars
   Documents retrieved: 4


💡 Key Insights:
   - All approaches provide relevant answers
   - MMR may provide more diverse context
   - HuggingFace embeddings are competitive and free
   - Choice depends on: budget, privacy needs, and quality requirements


---

## 9. Advanced Features

### 9.1 Custom Metadata Filtering

We added custom metadata earlier. Now let's use it to filter results.

In [24]:
# Example: Filter by source domain
print("Sample metadata from our documents:")
sample_doc = chunks[0]
print(f"\nMetadata: {sample_doc.metadata}")

# Create a retriever with metadata filter
filtered_retriever = vectorstore_openai.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 4,
        "filter": {"source_type": "web_documentation"}  # Filter by our custom metadata
    }
)

print("\n✓ Filtered retriever created")
print("  - Filter: source_type = 'web_documentation'")

# Test filtered retrieval
query = "What is a retriever in LangChain?"
filtered_docs = filtered_retriever.invoke(query)

print(f"\nQuery: '{query}'")
print(f"Retrieved {len(filtered_docs)} documents with metadata filter\n")

for i, doc in enumerate(filtered_docs[:2], 1):
    print(f"{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Source Type: {doc.metadata.get('source_type', 'N/A')}")
    print(f"   Process Date: {doc.metadata.get('process_date', 'N/A')}")
    print(f"   Content: {doc.page_content[:150]}...\n")

Sample metadata from our documents:

Metadata: {'source': 'https://python.langchain.com/docs/use_cases/question_answering/', 'title': 'Build a RAG agent with LangChain - Docs by LangChain', 'language': 'en', 'source_type': 'web_documentation', 'process_date': '2025-11-12', 'domain': 'langchain'}

✓ Filtered retriever created
  - Filter: source_type = 'web_documentation'

Query: 'What is a retriever in LangChain?'
Retrieved 4 documents with metadata filter

1. Source: https://python.langchain.com/docs/modules/data_connection/retrievers/
   Source Type: web_documentation
   Process Date: 2025-11-12
   Content: LangChain overview - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by...

2. Source: https://python.langchain.com/docs/modules/model_io/llms/
   Source Type: web_documentation
   Process Date: 2025-11-12
   Content: Models - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build th

### 9.2 Source Attribution

Show which sources were used to generate the answer.

### Production Optimizations

**Batch Processing for Better Performance:**

When processing many documents, use batch embedding operations instead of individual calls:

```python
# ❌ Inefficient: Individual embedding calls
for doc in docs:
    embedding = embeddings.embed_query(doc.page_content)
    
# ✓ Efficient: Batch embedding
texts = [doc.page_content for doc in docs]
embeddings_batch = embeddings.embed_documents(texts)  # Much faster!
```

**Performance Gains:**
- OpenAI API: Batch requests reduce latency overhead
- HuggingFace: GPU/CPU vectorization works better on batches
- Typical speedup: 2-5x faster for large document sets

**Memory Management:**

For large datasets (>10,000 documents):
- Use quantized FAISS indexes: `IndexIVFFlat` or `IndexIVFPQ`
- Consider production vector databases: Pinecone, Weaviate, Chroma
- Stream document processing instead of loading all at once

**Cost Optimization:**

```python
# Track OpenAI embedding costs
import tiktoken

def estimate_embedding_cost(texts, model="text-embedding-3-small"):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  # Approximation
    total_tokens = sum(len(encoding.encode(text)) for text in texts)
    cost_per_million = 0.02  # $0.02 per 1M tokens for text-embedding-3-small
    estimated_cost = (total_tokens / 1_000_000) * cost_per_million
    return total_tokens, estimated_cost

tokens, cost = estimate_embedding_cost([doc.page_content for doc in chunks])
print(f"Estimated tokens: {tokens:,}")
print(f"Estimated cost: ${cost:.4f}")
```

**Async Operations for Scale:**

```python
# For production systems handling many concurrent requests
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# Use async methods
async def embed_documents_async(texts):
    return await embeddings.aembed_documents(texts)

# Enables concurrent processing of multiple requests
```

In [25]:
query = "What are the key components of a RAG system?"

print(f"Query: {query}\n")
print("=" * 80)

# Get the documents first
docs_for_attribution = similarity_retriever.invoke(query)

# Get the answer
response = similarity_retrieval_chain.invoke(query)

print("\nANSWER:")
print("-" * 80)
print(response)

print("\n" + "=" * 80)
print("\nSOURCES USED:")
print("-" * 80)

# Extract unique sources from the retrieved documents
sources = set()
for doc in docs_for_attribution:
    source = doc.metadata.get('source', 'Unknown')
    sources.add(source)

for i, source in enumerate(sorted(sources), 1):
    print(f"{i}. {source}")

print("\n💡 Always cite sources to build trust and enable verification!")

Query: What are the key components of a RAG system?


ANSWER:
--------------------------------------------------------------------------------
The key components of a RAG (Retrieval-Augmented Generation) system, as outlined in the context, include:

1. **Indexing**: This is a pipeline for ingesting data from a source and indexing it, which usually occurs in a separate process.

2. **Retrieval and Generation**: This is the actual RAG process that takes the user query at runtime, retrieves the relevant data from the index, and then passes that data to the model for generation.

These components work together to enable the RAG system to effectively respond to user queries by combining retrieval of relevant information with generative capabilities.


SOURCES USED:
--------------------------------------------------------------------------------
1. https://python.langchain.com/docs/use_cases/chatbots/
2. https://python.langchain.com/docs/use_cases/question_answering/

💡 Always cite sources t

---

## 10. Best Practices

### Best Practices

1. **Chunk Size Selection**
   - Smaller chunks (300-500): Better for precise information retrieval
   - Larger chunks (800-1200): Better for context-heavy questions
   - Always use overlap (100-200 chars) to preserve context

2. **Embedding Selection**
   - **OpenAI**: Best quality, suitable for production, requires API key
   - **HuggingFace**: Free, private, good for development and privacy-sensitive data
   - Test both with your specific use case

3. **Retrieval Strategy**
   - **Similarity**: Use for most cases, simple and effective
   - **MMR**: Use when you need diverse results and want to avoid redundancy
   - Experiment with `k` (number of documents) - typically 3-5 is good

4. **Prompt Engineering**
   - Always instruct the model to say when it doesn't know
   - Request source citations for transparency
   - Be specific about the expected format

5. **Metadata Management**
   - Add custom metadata for filtering and attribution
   - Include source URLs, dates, document types
   - Use metadata for access control in production

